In [7]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

In [2]:
import os
PATH='data'
os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [3]:
train=pd.read_csv(f'{PATH}/subset.csv')
val=pd.read_csv(f'{PATH}/val_split.csv')

train.shape,val.shape

((2220, 8), (10932, 8))

In [4]:
BATCH_SIZE=64 #guarda en paquetes de 64

def decode(name,label):
    img=tf.io.read_file(name)#esto esta en byts
    img=tf.image.decode_jpeg(img,channels=3)#esto conviente en imagen
    img=tf.cast(img,tf.float32)/255#con esto hago que la imagen tenga precision de float32, Lo divido entre 255 qx yo quiero valores entre 0 y 1
    return img,label


def load_ds(df):
    imgs,labels=df["image_name"].values,df["target"].values #hacemos una lista con los nombre de las imagenes y su eqtiqueta 0 o 1
    imgs=[f'{PATH}/jpeg224/train/{name}.jpg' for name in imgs] #array donde tengo las rutas
    ds=tf.data.Dataset.from_tensor_slices((imgs,labels)) #ahora tiene el tensor o array los nombres de las imagenes y los labels
    ds=ds.map(decode)#el .map hace que a cada uno se le aplique una funcion
    ds=ds.shuffle(2048)#Con esto mescla cada ves que envia
    ds=ds.batch(BATCH_SIZE)
    return ds

In [5]:
train_ds=load_ds(train) #dataset de tensorflow
val_ds=load_ds(val) 

In [ ]:
#for img, label in train_ds.take(1):
#    print(img,label)

In [15]:
IMAGE_SIZE=(224,224,3)

encoder=ResNet50V2(
    include_top=False,#con esto le decimos que no queremos la ultima capa vamos a poner la nuestra    
    input_shape=IMAGE_SIZE,#Tamaño del input
    weights='imagenet' #el dataset que fue entrenado con esto los pesos ya saben detectar colores formas y otras cosas
)

encoder.trainable=False #no va a aplicar los gradientes a la red pero calcula
inputs = keras.Input(shape=IMAGE_SIZE)
#x=keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs) #en la version 2.3 se puede poner esto y borrar el 255 de arriba
#x=encoder(x,training=False)#con esto no calcula los gradientes
x=encoder(inputs,training=False)#con esto no calcula los gradientes
x=keras.layers.GlobalAveragePooling2D()(x)#lo que sale de la resnet, dice que le aplica algo y lo deja con el numero de canales en est caso 2048
outputs=keras.layers.Dense(1,activation="sigmoid")(x)#lo que sale de la capa lo va a poner entre 0 y 1 por el sigmoid
model=keras.Model(inputs,outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [17]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [18]:
model.fit(train_ds,epochs=10,validation_data=val_ds,validation_steps=10)

Train for 35 steps, validate for 5 steps
Epoch 1/5
 1/35 [..............................] - ETA: 27:35

KeyboardInterrupt: 

In [19]:
test=pd.read_csv(f'{PATH}/test.csv')
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [ ]:
BATCH_SIZE=256 
def decode_test(name):
    img=tf.io.read_file(name)#esto esta en byts
    img=tf.image.decode_jpeg(img,channels=3)#esto conviente en imagen
    img=tf.cast(img,tf.float32)/255#con esto hago que la imagen tenga precision de float32, Lo divido entre 255 qx yo quiero valores entre 0 y 1
    return img


def load_test_ds(df):
    imgs=df["image_name"].values #hacemos una lista con los nombre de las imagenes y su eqtiqueta 0 o 1
    imgs=[f'{PATH}/jpeg224/test/{name}.jpg' for name in imgs] #array donde tengo las rutas
    ds=tf.data.Dataset.from_tensor_slices(imgs) #ahora tiene el tensor o array los nombres de las imagenes y los labels
    ds=ds.map(decode_test)#el .map hace que a cada uno se le aplique una funcion
    ds=ds.batch(BATCH_SIZE)
    return ds

In [ ]:
test_ds=load_test_ds(test)

In [ ]:
preds=model.predict(test_ds)

In [ ]:
preds

In [ ]:
submission=pd.DataFrame({
    "image_name":test["image_name"].values,
    "target":preds.ravel()#deja la dimension en 1
    
})

submission.head()

In [ ]:
submission.to_csv("submission.csv",index=False)